# Compile & Run Enformer on Inf2

In [ ]:
# patch required to make the neuronx compilation work
!pygmentize neuron.patch

In [ ]:
import os

if not os.path.isdir('enformer-pytorch'):
    !git clone -b 0.7.6 https://github.com/lucidrains/enformer-pytorch && \
    cd enformer-pytorch && git apply ../neuron.patch

In [ ]:
import os
import urllib.request

if not os.path.isfile("test-sample.pt"):
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/lucidrains/enformer-pytorch/main/data/test-sample.pt",
        "test-sample.pt"
    )

In [ ]:
import sys
sys.path.insert(0, "enformer-pytorch")
import torch
from enformer_pytorch import Enformer

model = Enformer.from_pretrained('EleutherAI/enformer-official-rough', return_dict=False)
model.eval()

data = torch.load('test-sample.pt', map_location=torch.device('cpu'))
seq, target = data['sequence'].unsqueeze(0), data['target'].unsqueeze(0)

In [ ]:
import random
import numpy as np
from enformer_pytorch.data import str_to_one_hot

# this is the expected seq len of the model
SEQUENCE_LENGTH=196608

def _get_random_input():
    seq = "".join(
        [random.choice("ACGT") for _ in range(SEQUENCE_LENGTH)])
    return torch.from_numpy(np.expand_dims(str_to_one_hot(seq), 0).astype(np.float32)),seq
inputs,_ = _get_random_input()

In [ ]:
# remove dictionaries
import types
if not hasattr(model, 'forward_'): model.forward_ = model.forward
model.forward = types.MethodType(lambda self,x: tuple(self.forward_(x).values()), model)

In [ ]:
# warm up model
out = model(inputs)

In [ ]:
# compile and save
%%time
import torch_neuronx
with torch.no_grad():  
    #model_support = torch_neuronx.analyze(model, inputs)
    neuron_model = torch_neuronx.trace(
         model, inputs,
#         compiler_args=[
#             "--auto-cast", "none"
#             "--verbose", "debug",
#         ]
     )
    neuron_model.save("neuron_model.pt")

In [ ]:
neuron_model = torch.jit.load("neuron_model.pt")

In [ ]:
# create a padded input with the test values
inputs2 = torch.zeros(inputs.shape)
inputs2[:, :seq.shape[1], :] = seq[:]

In [ ]:
with torch.no_grad():
    out1 = model(inputs2)
    out2 = neuron_model(inputs2)

In [9]:
from enformer_pytorch.modeling_enformer import pearson_corr_coef

print(f"Correlation - class 1 human - CPU: {pearson_corr_coef(out1[0],target)}, Neuron: {pearson_corr_coef(out2[0],target)}")

Correlation - class 1 human - CPU: tensor([-0.0305]), Neuron: tensor([-0.0325])


In [10]:
pearson_corr_coef(out1[0],out2[0]), out1[0],out2[0]

(tensor([0.7890]),
 tensor([[[0.0739, 0.0826, 0.0836,  ..., 0.0043, 0.0348, 0.0100],
          [0.0748, 0.0844, 0.0911,  ..., 0.0027, 0.0364, 0.0099],
          [0.0862, 0.1005, 0.1166,  ..., 0.0030, 0.0402, 0.0096],
          ...,
          [0.0038, 0.0037, 0.0028,  ..., 0.0037, 0.0488, 0.0098],
          [0.0038, 0.0036, 0.0028,  ..., 0.0037, 0.0489, 0.0097],
          [0.0038, 0.0036, 0.0028,  ..., 0.0037, 0.0488, 0.0097]]]),
 tensor([[[-2.2772, -2.2545, -2.1328,  ..., -4.8374, -2.7688, -4.2377],
          [-2.3011, -2.2586, -2.0646,  ..., -5.3634, -2.8254, -4.2367],
          [-2.0613, -2.0571, -1.7767,  ..., -5.3323, -2.7203, -4.3290],
          ...,
          [-5.1802, -5.1457, -5.1899,  ..., -5.9974, -3.4966, -5.3061],
          [-5.1802, -5.1457, -5.1899,  ..., -5.9974, -3.4966, -5.3061],
          [-5.1802, -5.1457, -5.1899,  ..., -5.9974, -3.4966, -5.3061]]]))

In [11]:
import time
it=10
with torch.no_grad():
    t=time.time()
    for i in range(it):
        out1 = model(inputs2)
    print(f"Elapsed CPU: {(time.time()-t) * 1000 / it}")
    t=time.time()
    for i in range(it):
        out2 = neuron_model(inputs2)
    print(f"Elapsed Inf2: {(time.time()-t) * 1000 / it}")

Elapsed CPU: 5049.9802350997925
Elapsed Inf2: 583.141565322876
